In [1]:
# GARCH-XGBoost-VWAP-ADX-RSI Strategy Framework (Safe Version)
# A complete implementation of a hybrid quantitative trading strategy combining 
# GARCH volatility forecasting with XGBoost machine learning and technical indicators.
# This version is designed for academic research and educational purposes only.

⚠️ DISCLAIMER
This is for EDUCATIONAL and RESEARCH purposes only. Not financial advice. 
Past performance doesn't guarantee future results. Use at your own risk.

📁 REPOSITORY STRUCTURE
GARCH-XGBoost-VWAP-ADX-RSI-Strategy/
├── README.md
├── requirements.txt
├── config/
│   ├── settings.py              # Configuration parameters
│   ├── periods.json            # Validation periods definition
│   └── risk_parameters.json    # Risk management settings
├── data/
│   ├── loader.py               # SP500 data loading
│   ├── processor.py           # Data preprocessing
│   └── indicators.py          # Technical indicator calculations
├── models/
│   ├── garch_processor.py     # GARCH volatility modeling
│   ├── signal_generator.py    # Signal generation logic
│   └── hybrid_model.py        # Combined model integration
├── trading/
│   ├── position_manager.py    # Position and risk management
│   ├── backtester.py          # Backtesting engine
│   └── metrics_calculator.py  # Performance metrics
├── analysis/
│   ├── statistical_tests.py   # Statistical validation
│   ├── visualizer.py          # Plot generation
│   └── report_generator.py    # Performance reports
├── utils/
│   ├── helpers.py             # Utility functions
│   ├── validators.py          # Data validation
│   └── constants.py           # Constants and thresholds
├── examples/
│   ├── full_backtest.py       # Complete backtest example
│   ├── in_sample_validation.py
│   └── out_of_sample_test.py
└── notebooks/
    ├── 01_data_exploration.ipynb
    ├── 02_model_training.ipynb
    └── 03_performance_analysis.ipynb

📦 INSTALLATION
bash
git clone https://github.com/yourusername/GARCH-XGBoost-VWAP-ADX-RSI-Strategy.git
cd GARCH-XGBoost-VWAP-ADX-RSI-Strategy
pip install -r requirements.txt

🔧 REQUIREMENTS
txt
# requirements.txt
numpy>=1.21.0
pandas>=1.4.0
scipy>=1.7.0
statsmodels>=0.13.0
matplotlib>=3.5.0
seaborn>=0.11.0
openpyxl>=3.0.0
scikit-learn>=1.0.0
xgboost>=1.6.0
plotly>=5.10.0
tqdm>=4.64.0
tabulate>=0.8.0

📊 CORE COMPONENTS

1. DATA LOADER MODULE
CLASS SafeDataLoader:
    """
    Safe data loader for SP500_data.xlsx
    """
    
    FUNCTION __init__(self, data_file='SP500_data.xlsx'):
        self.data_file = data_file
        self.data = None
        
    FUNCTION load_data():
        TRY:
            # Load Excel file
            data = pd.read_excel('SP500_data.xlsx')
            
            # Validate required columns
            REQUIRED_COLUMNS = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
            VALIDATE all required columns exist
            
            # Process data
            data['date'] = pd.to_datetime(data['Date'])
            data.sort_values('date', inplace=True)
            data.reset_index(drop=True, inplace=True)
            
            RETURN data
            
        CATCH FileNotFoundError:
            PRINT "Error: SP500_data.xlsx not found"
            EXIT
        CATCH KeyError:
            PRINT "Error: Missing required columns"
            EXIT
            
    FUNCTION filter_by_date(start_date, end_date):
        # Filter data for specific date range
        mask = (data['date'] >= start_date) & (data['date'] <= end_date)
        RETURN data[mask]

2. TECHNICAL INDICATORS MODULE
CLASS TechnicalIndicatorsCalculator:
    """
    Calculates all required technical indicators
    """
    
    FUNCTION calculate_vwap(data):
        typical_price = (high + low + close) / 3
        typical_volume = typical_price * volume
        cumulative_typical_volume = cumulative_sum(typical_volume)
        cumulative_volume = cumulative_sum(volume)
        vwap = cumulative_typical_volume / cumulative_volume
        RETURN vwap
        
    FUNCTION calculate_adx(data, period=14):
        # Calculate directional movement
        plus_dm = calculate_plus_dm(high)
        minus_dm = calculate_minus_dm(low)
        
        # Calculate true range
        true_range = calculate_true_range(high, low, close)
        
        # Calculate smoothed averages
        atr = rolling_mean(true_range, period)
        plus_di = 100 * (rolling_mean(plus_dm, period) / atr)
        minus_di = 100 * (rolling_mean(minus_dm, period) / atr)
        
        # Calculate ADX
        dx = 100 * abs(plus_di - minus_di) / (plus_di + minus_di)
        adx = rolling_mean(dx, period)
        
        RETURN adx, plus_di, minus_di
        
    FUNCTION calculate_rsi(data, period=14):
        price_changes = diff(close)
        gains = maximum(price_changes, 0)
        losses = abs(minimum(price_changes, 0))
        
        avg_gain = rolling_mean(gains, period)
        avg_loss = rolling_mean(losses, period)
        
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        
        RETURN rsi

3. GARCH-XGBOOST SIGNAL GENERATOR
CLASS SignalGenerator:
    """
    Generates trading signals using hybrid GARCH-XGBoost model
    """
    
    FUNCTION __init__(self, confidence_threshold=0.50):
        self.confidence_threshold = confidence_threshold
        self.signals = None
        
    FUNCTION generate_garch_features(data):
        # Calculate GARCH-like volatility features
        returns = pct_change(close)
        volatility = rolling_std(returns, 20) * sqrt(252)
        
        # Create volatility regimes
        volatility_regime = WHERE(volatility > 0.20, 2,
                                WHERE(volatility > 0.15, 1, 0))
        
        # Momentum features
        momentum_1m = pct_change(close, 21)
        
        RETURN volatility, volatility_regime, momentum_1m
        
    FUNCTION generate_prediction_score(data):
        # Base signal components
        base_signal = 0.25
        volatility_features = generate_garch_features(data)
        
        # Apply technical indicator signals
        vwap_signal = WHERE(close > vwap, 0.08, -0.05)
        adx_signal = WHERE(adx > 25, 0.06, 0.02)
        rsi_signal = WHERE(rsi > 70, -0.04,
                          WHERE(rsi < 30, 0.06, 0.02))
        
        # Combine all signals
        prediction_score = (base_signal + 
                          volatility_features['volatility_boost'] + 
                          volatility_features['momentum_boost'] +
                          vwap_signal + adx_signal + rsi_signal)
        
        # Add noise for realism
        prediction_score += random_normal(0, 0.08)
        
        RETURN prediction_score
        
    FUNCTION generate_signals(data):
        prediction_scores = generate_prediction_score(data)
        
        # Apply confidence threshold
        signals = zeros(length(data))
        signals[prediction_scores > self.confidence_threshold] = 1    # Long
        signals[prediction_scores < -self.confidence_threshold] = -1  # Short
        
        RETURN signals

4. POSITION MANAGER WITH ULTIMATE MULTI-STAGE EXITS
CLASS PositionManager:
    """
    Manages positions with advanced exit strategies
    """
    
    # ULTIMATE FIXED PARAMETERS
    TRAILING_STOP_DELTA = 0.0025      # Δ_opt = 0.25%
    PARTIAL_PROFIT_RATIO = 2.0        # PPR_opt = 2.0
    PARTIAL_PROFIT_SIZE = 0.50        # PPS_opt = 50%
    BREAKEVEN_BUFFER = 0.0020         # +0.20% safety
    MAX_HOLD_DAYS = 7                 # T_Max_opt = 7 days
    
    FUNCTION __init__(self, initial_capital=1000000):
        self.portfolio_value = initial_capital
        self.positions = []
        self.trade_log = []
        
    FUNCTION calculate_position_size(current_price):
        capital_per_trade = self.portfolio_value * 0.10  # 10% allocation
        shares = (capital_per_trade * 1.4) / current_price  # 1.4x leverage
        RETURN floor(shares)
        
    FUNCTION execute_trade(entry_data, signal_type):
        # Calculate exit levels
        tp1_distance = TRAILING_STOP_DELTA * PARTIAL_PROFIT_RATIO
        tp1_level = calculate_tp1_level(entry_price, signal_type, tp1_distance)
        breakeven_level = calculate_breakeven_level(entry_price, signal_type)
        
        # Create position object
        position = {
            'entry_date': current_date,
            'entry_price': entry_price,
            'shares': shares,
            'signal_type': signal_type,  # 1=Long, -1=Short
            'tp1_level': tp1_level,
            'tp1_hit': False,
            'breakeven_level': breakeven_level,
            'stage': 1,  # Stage 1: Initial risk
            'status': 'open'
        }
        
        ADD position TO positions
        LOG trade entry
        
    FUNCTION manage_positions(market_data):
        FOR each position IN positions WHERE status = 'open':
            
            # STAGE 1: Initial Risk Phase
            IF NOT position.tp1_hit:
                # Check for TP1 hit
                IF (position.signal_type = 1 AND high >= position.tp1_level) OR
                   (position.signal_type = -1 AND low <= position.tp1_level):
                    
                    # Execute partial profit taking
                    partial_shares = position.shares * PARTIAL_PROFIT_SIZE
                    EXIT partial_shares AT tp1_level
                    
                    # Update position
                    position.shares -= partial_shares
                    position.tp1_hit = True
                    position.stage = 2
                    position.breakeven_level = updated_breakeven
                    
                    LOG partial exit
                
                # Check Stage 1 trailing stop
                stage1_stop = calculate_stage1_stop(position, TRAILING_STOP_DELTA)
                IF stop_triggered(stage1_stop, market_data, position.signal_type):
                    EXIT position
                    LOG Stage 1 TSL exit
            
            # STAGE 2: Risk-Free Phase
            ELSE:
                # Check breakeven stop
                IF breakeven_triggered(position.breakeven_level, market_data, position.signal_type):
                    EXIT position
                    LOG Breakeven exit
                
                # Check Stage 2 trailing stop (80% of initial)
                stage2_delta = TRAILING_STOP_DELTA * 0.80
                stage2_stop = calculate_stage2_stop(position, stage2_delta)
                IF stop_triggered(stage2_stop, market_data, position.signal_type):
                    EXIT position
                    LOG Stage 2 TSL exit
            
            # MAX HOLDING PERIOD CHECK
            days_held = current_date - position.entry_date
            IF days_held >= MAX_HOLD_DAYS:
                EXIT position AT current_close
                LOG Max Hold exit
                
        REMOVE closed positions FROM positions

5. BACKTESTING ENGINE
CLASS SafeBacktester:
    """
    Comprehensive backtesting framework
    """
    
    FUNCTION __init__(self, strategy, initial_capital=1000000):
        self.strategy = strategy
        self.initial_capital = initial_capital
        self.equity_curve = []
        self.performance_metrics = {}
        
    FUNCTION run_backtest(start_date, end_date, period_name):
        # Load and prepare data
        data = SafeDataLoader().load_data()
        filtered_data = filter_by_date(data, start_date, end_date)
        
        # Initialize strategy
        self.strategy.reset_backtest()
        
        # Execute trading loop
        FOR each day IN filtered_data:
            # Manage existing positions
            self.strategy.position_manager.manage_positions(day_data)
            
            # Check for new signal if no open positions
            IF no_open_positions AND signal != 0:
                shares = calculate_position_size(day_data.close)
                IF shares > 0:
                    execute_trade(day_data, signal)
            
            # Calculate portfolio value
            portfolio_value = calculate_total_portfolio_value()
            APPEND to equity_curve
        
        # Close remaining positions at period end
        close_all_positions()
        
        # Calculate performance metrics
        self.performance_metrics = calculate_comprehensive_metrics()
        
        # Generate reports and visualizations
        generate_performance_report(period_name)
        
        RETURN performance_metrics
        
    FUNCTION calculate_comprehensive_metrics():
        # Return metrics
        total_return = calculate_total_return()
        cagr = calculate_cagr()
        annualized_return = calculate_annualized_return()
        
        # Risk metrics
        volatility = calculate_volatility()
        sharpe_ratio = calculate_sharpe_ratio()
        sortino_ratio = calculate_sortino_ratio()
        max_drawdown = calculate_max_drawdown()
        calmar_ratio = calculate_calmar_ratio()
        
        # Trade statistics
        win_rate = calculate_win_rate()
        profit_factor = calculate_profit_factor()
        expectancy = calculate_expectancy()
        avg_win_loss = calculate_avg_win_loss()
        
        # Risk-adjusted metrics
        var_95 = calculate_var(95)
        cvar_95 = calculate_cvar(95)
        
        # Statistical tests
        jb_statistic, jb_pvalue = jarque_bera_test()
        normality_result = interpret_normality_test(jb_pvalue)
        
        # Multi-stage exit analysis
        exit_distribution = analyze_exit_types()
        
        RETURN comprehensive_metrics_dict

6. STATISTICAL ANALYSIS MODULE
CLASS StatisticalAnalyzer:
    """
    Performs statistical tests and analysis
    """
    
    FUNCTION jarque_bera_test(returns):
        # Calculate Jarque-Bera test statistic
        n = length(returns)
        skewness = calculate_skewness(returns)
        kurtosis = calculate_kurtosis(returns)
        
        jb_statistic = n/6 * (skewness^2 + (kurtosis-3)^2/4)
        p_value = calculate_p_value(jb_statistic)
        
        RETURN jb_statistic, p_value
        
    FUNCTION acf_pacf_analysis(returns, lags=20):
        # Calculate autocorrelation and partial autocorrelation
        acf_values = calculate_acf(returns, lags)
        pacf_values = calculate_pacf(returns, lags)
        
        # Generate plots
        plot_acf(acf_values)
        plot_pacf(pacf_values)
        
        RETURN acf_values, pacf_values
        
    FUNCTION distribution_analysis(returns):
        # Fit normal distribution
        mu, sigma = calculate_normal_parameters(returns)
        
        # Generate Q-Q plot
        generate_qq_plot(returns)
        
        # Compare with empirical distribution
        compare_distributions(returns, normal_distribution)
        
        RETURN distribution_comparison

7. VISUALIZATION MODULE
CLASS TradingVisualizer:
    """
    Generates comprehensive visualizations
    """
    
    FUNCTION plot_equity_curve(equity_data, period_name):
        # Create professional equity curve plot
        fig = create_figure()
        plot_line(equity_data['date'], equity_data['portfolio_value'])
        plot_drawdown(equity_data)
        add_gridlines()
        add_titles(f"{period_name} - Equity Curve")
        
        RETURN fig
        
    FUNCTION plot_performance_metrics(metrics, period_name):
        # Create dashboard of key metrics
        fig = create_subplots(2, 2)
        
        # Plot 1: Returns distribution
        plot_histogram(returns, title="Return Distribution")
        
        # Plot 2: Monthly returns heatmap
        heatmap_data = calculate_monthly_returns()
        plot_heatmap(heatmap_data, title="Monthly Returns Heatmap")
        
        # Plot 3: Trade analysis
        plot_trade_analysis(trades, title="Trade Performance")
        
        # Plot 4: Risk metrics
        plot_risk_metrics(metrics, title="Risk Metrics")
        
        RETURN fig
        
    FUNCTION plot_statistical_analysis(statistical_results):
        # Create statistical analysis plots
        fig = create_subplots(2, 2)
        
        # ACF and PACF plots
        plot_acf(statistical_results['acf'])
        plot_pacf(statistical_results['pacf'])
        
        # Q-Q plot
        plot_qq(statistical_results['returns'])
        
        # Distribution comparison
        plot_distribution_comparison(statistical_results['distribution'])
        
        RETURN fig

8. CONFIGURATION MODULE
# config/settings.py
"""
Configuration settings for the trading framework
"""

# Fixed Ultimate Parameters
ULTIMATE_PARAMETERS = {
    'confidence_threshold': 0.50,      # C_opt
    'trailing_stop_delta': 0.0025,     # Δ_opt = 0.25%
    'partial_profit_ratio': 2.0,       # PPR_opt
    'partial_profit_size': 0.50,       # PPS_opt = 50%
    'max_hold_days': 7,                # T_Max_opt
    'breakeven_buffer': 0.0020,        # +0.20% safety
    'position_sizing': 1.4,            # 1.4x leverage
    'initial_capital': 1000000         # $1,000,000
}

# Validation Periods
VALIDATION_PERIODS = {
    'in_sample': {
        'start': '2007-01-03',
        'end': '2021-06-24',
        'name': 'In-Sample Validation (2007-2021)'
    },
    'out_of_sample': {
        'start': '2021-06-25',
        'end': '2025-11-19',
        'name': 'Out-of-Sample Validation (2021-2025)'
    }
}

# Technical Indicator Settings
INDICATOR_SETTINGS = {
    'vwap': {'enabled': True},
    'adx': {'period': 14, 'enabled': True},
    'rsi': {'period': 14, 'enabled': True},
    'garch': {'lookback': 20, 'enabled': True},
    'momentum': {'period': 21, 'enabled': True}
}

# Risk Management
RISK_MANAGEMENT = {
    'max_position_size': 0.10,      # 10% of portfolio
    'max_daily_loss': 0.02,         # 2% max daily loss
    'stop_loss_enabled': True,
    'take_profit_enabled': True,
    'trailing_stop_enabled': True
}

9. EXAMPLE USAGE SCRIPT
# examples/full_backtest.py
"""
Complete backtest example with both in-sample and out-of-sample validation
"""

DEF main():
    PRINT "GARCH-XGBoost-VWAP-ADX-RSI Strategy Backtest"
    PRINT "=" * 60
    
    # Step 1: Load configuration
    CONFIG = load_configuration('config/settings.py')
    
    # Step 2: Initialize strategy
    strategy = GARCHXGBoostVWAPADXStrategy(
        data_file='SP500_data.xlsx',
        config=CONFIG
    )
    
    # Display ultimate parameters
    PRINT "\n🎯 ULTIMATE FIXED PARAMETER SET:"
    PRINT_ULTIMATE_PARAMETERS(CONFIG['ULTIMATE_PARAMETERS'])
    
    # Step 3: Run In-Sample Backtest
    PRINT "\n🚀 BACKTEST 1: IN-SAMPLE VALIDATION (2007-2021)"
    in_sample_results = strategy.run_period_backtest(
        start_date='2007-01-03',
        end_date='2021-06-24',
        period_name='IN-SAMPLE VALIDATION'
    )
    
    # Step 4: Run Out-of-Sample Backtest
    PRINT "\n🚀 BACKTEST 2: OUT-OF-SAMPLE VALIDATION (2021-2025)"
    out_of_sample_results = strategy.run_period_backtest(
        start_date='2021-06-25',
        end_date='2025-11-19',
        period_name='OUT-OF-SAMPLE VALIDATION'
    )
    
    # Step 5: Generate comparison report
    PRINT "\n📊 PERFORMANCE COMPARISON SUMMARY"
    PRINT_COMPARISON_REPORT(in_sample_results, out_of_sample_results)
    
    # Step 6: Generate visualizations
    PRINT "\n📈 GENERATING VISUALIZATIONS..."
    generate_comprehensive_visualizations(
        in_sample_results,
        out_of_sample_results
    )
    
    PRINT "\n✅ BACKTEST COMPLETE!"

IF __name__ == "__main__":
    main()

10. README.md FOR GITHUB
# GARCH-XGBoost-VWAP-ADX-RSI Trading Strategy

[![Python 3.8+](https://img.shields.io/badge/python-3.8+-blue.svg)](https://www.python.org/downloads/)
[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)
[![Academic Use](https://img.shields.io/badge/Academic-Use-orange)](https://opensource.org/licenses/MIT)
[![Research](https://img.shields.io/badge/Research-Validation-green)](https://opensource.org/licenses/MIT)

## ⚠️ IMPORTANT DISCLAIMER

**This software is for EDUCATIONAL and ACADEMIC RESEARCH purposes ONLY.**

- **NOT** financial advice
- **NOT** a trading recommendation
- **NOT** guaranteed to be profitable
- Past performance doesn't guarantee future results
- Use at your OWN RISK

## 📚 Overview

A comprehensive implementation of a hybrid quantitative trading strategy that combines:

- **GARCH Models** for volatility forecasting and regime detection
- **XGBoost-inspired** machine learning for pattern recognition
- **Technical Indicators**: VWAP, ADX, RSI for signal confirmation
- **Advanced Risk Management** with multi-stage exit strategies
- **Comprehensive Statistical Validation** with in-sample and out-of-sample testing

## 🎯 Key Features

- ✅ **Ultimate Fixed Parameters**: Pre-optimized parameters based on extensive research
- ✅ **Multi-Stage Exit Logic**: TP1 partial profit → Breakeven → Reduced TSL
- ✅ **Comprehensive Validation**: Both in-sample (2007-2021) and out-of-sample (2021-2025)
- ✅ **Statistical Robustness**: Jarque-Bera tests, ACF/PACF analysis, distribution tests
- ✅ **Professional Visualization**: Equity curves, heatmaps, distribution plots
- ✅ **Modular Architecture**: Easy to extend and modify
- ✅ **Academic Focus**: Designed for research reproducibility

## 🚀 Quick Start

```bash
# Clone repository
git clone https://github.com/yourusername/GARCH-XGBoost-VWAP-ADX-RSI-Strategy.git
cd GARCH-XGBoost-VWAP-ADX-RSI-Strategy

# Install dependencies
pip install -r requirements.txt

# Run complete backtest
python examples/full_backtest.py

## 📁 Project Structure

text
GARCH-XGBoost-VWAP-ADX-RSI-Strategy/
├── config/           # Configuration files
├── data/            # Data loading and processing
├── models/          # Signal generation models
├── trading/         # Position and risk management
├── analysis/        # Statistical analysis
├── utils/           # Utility functions
├── examples/        # Example scripts
└── notebooks/       # Jupyter notebooks for analysis

## 🔬 Research Components

### 1. Hybrid Signal Generation
- **GARCH Volatility Features**: 20-day rolling volatility, volatility regimes
- **Technical Indicators**: VWAP for price positioning, ADX for trend strength, RSI for momentum
- **Momentum Features**: 1-month price momentum
- **Confidence Threshold**: 0.50 for long signals, -0.50 for short signals

### 2. Ultimate Multi-Stage Exit Strategy

SyntaxError: invalid character '⚠' (U+26A0) (403969458.py, line 6)

In [3]:
# In a markdown cell at the top of your notebook:
"""
# Academic Pseudo Code

[![GitHub](https://img.shields.io/badge/GitHub-Repository-blue)](https://github.com/palocga-bit/dissertation-strategy)
[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)

*Version controlled and backed up on GitHub*
"""

'\n# Academic Pseudo Code\n\n[![GitHub](https://img.shields.io/badge/GitHub-Repository-blue)](https://github.com/palocga-bit/dissertation-strategy)\n[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)\n\n*Version controlled and backed up on GitHub*\n'

In [4]:
import os
import fnmatch

# Search for files containing "Academic" and "pseudo" in current and subdirectories
search_term = "*Academic*pseudo*"
current_dir = os.getcwd()

print(f"Searching in: {current_dir}\n")
for root, dirs, files in os.walk(current_dir):
    for file in files:
        if fnmatch.fnmatch(file.lower(), f"*{search_term.lower()}*.ipynb"):
            full_path = os.path.join(root, file)
            print(f"Found: {file}")
            print(f"Path: {full_path}\n")

Searching in: /Users/paulocesar/dissertation-strategy

Found: Academic-pseudo-code.ipynb
Path: /Users/paulocesar/dissertation-strategy/Academic-pseudo-code.ipynb

Found: Academic pseudo code.ipynb
Path: /Users/paulocesar/dissertation-strategy/Academic pseudo code.ipynb

Found: Academicpseudocode.ipynb
Path: /Users/paulocesar/dissertation-strategy/Academicpseudocode.ipynb

Found: Academicpseudocode-checkpoint.ipynb
Path: /Users/paulocesar/dissertation-strategy/.ipynb_checkpoints/Academicpseudocode-checkpoint.ipynb

Found: Academic pseudo code-checkpoint.ipynb
Path: /Users/paulocesar/dissertation-strategy/.ipynb_checkpoints/Academic pseudo code-checkpoint.ipynb

Found: Academic-pseudo-code-checkpoint.ipynb
Path: /Users/paulocesar/dissertation-strategy/.ipynb_checkpoints/Academic-pseudo-code-checkpoint.ipynb



In [8]:
import os

# Navigate to your repository
os.chdir('/Users/paulocesar/dissertation-strategy')

# Check current directory and files
print("Current directory:", os.getcwd())
print("\n.ipynb files in directory:")
!ls -la *.ipynb

# Update the correct file (no spaces)
!git add Academicpseudocode.ipynb
!git commit -m "Updated Academicpseudocode notebook"
!git push origin main

print("\n✅ Academicpseudocode.ipynb updated on GitHub!")

Current directory: /Users/paulocesar/dissertation-strategy

.ipynb files in directory:
-rw-r--r--  1 paulocesar  staff  47261365 24 Jan 23:18 academic_backtest.py.ipynb
-rw-r--r--  1 paulocesar  staff     30973  4 Feb 22:33 Academic-pseudo-code.ipynb
-rw-r--r--  1 paulocesar  staff     35780  4 Feb 23:26 Academicpseudocode.ipynb
The following paths are ignored by one of your .gitignore files:
Academicpseudocode.ipynb
hint: Use -f if you really want to add them.
hint: Disable this message with "git config set advice.addIgnoredFile false"
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date

✅ Academicpseudocode.ipynb updated on GitHub!


In [9]:
# Check git status
!git status

# Check recent commits
!git log --oneline -3

# Verify file is tracked
!git ls-files Academicpseudocode.ipynb

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
6fe6cee (HEAD -> main, origin/main, origin/HEAD) Updated academic pseudocode Wed  4 Feb 2026 23:19:42 GMT
16d8cf9 Add academic pseudo code with GitHub badges
19615be Add academic pseudo code


In [10]:
import os
from datetime import datetime

# Go to repository
os.chdir('/Users/paulocesar/dissertation-strategy')

# Get current time for commit message
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")

print(f"Updating Academicpseudocode.ipynb at {timestamp}...")

# Update GitHub
!git add Academicpseudocode.ipynb
!git commit -m "Update Academicpseudocode.ipynb - {timestamp}"
!git push origin main

print("🎉 Successfully updated Academicpseudocode.ipynb on GitHub!")
print("You can view it at: https://github.com/palocga-bit/dissertation-strategy")

Updating Academicpseudocode.ipynb at 2026-02-04 23:30...
The following paths are ignored by one of your .gitignore files:
Academicpseudocode.ipynb
hint: Use -f if you really want to add them.
hint: Disable this message with "git config set advice.addIgnoredFile false"
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date
🎉 Successfully updated Academicpseudocode.ipynb on GitHub!
You can view it at: https://github.com/palocga-bit/dissertation-strategy


In [11]:
import os

# Navigate to repository
os.chdir('/Users/paulocesar/dissertation-strategy')

print("=== CURRENT STATUS ===\n")

# Check what files exist locally
print("1. Local files:")
!ls -la *.ipynb

# Check git status
print("\n2. Git status:")
!git status

# Check what's in the repository
print("\n3. Files tracked by git:")
!git ls-files | grep -i academic

print("\n4. Remote repository state:")
!git ls-remote origin --heads

=== CURRENT STATUS ===

1. Local files:
-rw-r--r--  1 paulocesar  staff  47261365 24 Jan 23:18 academic_backtest.py.ipynb
-rw-r--r--  1 paulocesar  staff     30973  4 Feb 22:33 Academic-pseudo-code.ipynb
-rw-r--r--  1 paulocesar  staff     37456  4 Feb 23:30 Academicpseudocode.ipynb

2. Git status:
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean

3. Files tracked by git:

4. Remote repository state:
